In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time

In [2]:
bulktrade_basedir = './국토교통부_실거래가_공개시스템/집값분석/일괄구매/10호이상/'

In [3]:
%%time
missed_df = pd.read_csv(bulktrade_basedir+'빠진애들.csv')
print(missed_df.shape)
missed_df.head()

(2097, 15)
Wall time: 17 ms


,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동
0,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,23.55,17000,5,2016.0,2.0,2018-01-04,721.868365,5.0,오피스텔,NaN,강남구,세곡동
1,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,5,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,NaN,강남구,세곡동
2,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,6,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,NaN,강남구,세곡동
3,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,8,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,NaN,강남구,세곡동
4,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,5,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,NaN,강남구,세곡동


In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [22]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\max6296\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [24]:
driver.get("http://www.iros.go.kr/PMainJ.jsp")

In [37]:
driver.get("http://www.iros.go.kr/frontservlet?cmd=RISUWelcomeViewC")

In [38]:
def get_owner_name(df):
    df = df.copy()
    
    count = 0
    for addr in tqdm(df['지번주소'].unique()):
        owner_name = np.nan
        
        addr_df = df[df['지번주소'] == addr]
        addr = addr.replace('-0', '')
        
        driver.switch_to.default_content()
        
        
        driver.switch_to.frame("inputFrame")
        
        elem_search = driver.find_element_by_xpath('html/body/form[1]/div[24]/div/div/div/fieldset/div/table/tbody/tr[5]/td/span/input')
        elem_search.clear()
        elem_search.send_keys(addr)
        elem_search.send_keys(Keys.RETURN)
        time.sleep(2)
        
        try:
            driver.switch_to.alert.accept()
        except:
            pass
        
        time.sleep(1)
        
        driver.switch_to.default_content()
        driver.switch_to.frame('resultFrame')
        driver.switch_to.frame('frmOuterModal')
        
        time.sleep(1)
        
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr[3]/td[6]/button').click()
        except:
            continue
        
        time.sleep(1)
        
        elem_result = driver.find_element_by_xpath('/html/body/div[2]/div[2]/table/tbody/tr/td[3]')
        
        owner_name = elem_result.text
        
        print(addr)
        print(owner_name)
        
        driver.switch_to.default_content()
        
        df.loc[addr_df.index, '소유자'] = owner_name
        
        count += 1
    
    return df

In [39]:
prac_df = get_owner_name(missed_df)

  1%|▉                                                                                  | 1/85 [00:05<07:30,  5.37s/it]

서울특별시 강남구 세곡동 582
주식회사하***


  2%|█▉                                                                                 | 2/85 [00:10<07:21,  5.31s/it]

서울특별시 강남구 세곡동 586
곽**


  4%|██▉                                                                                | 3/85 [00:15<07:14,  5.29s/it]

...

서울특별시 중랑구 상봉동 311
한***~


 99%|█████████████████████████████████████████████████████████████████████████████████ | 84/85 [07:24<00:05,  5.13s/it]

서울특별시 중랑구 중화동 10
한***~


100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [07:30<00:00,  5.30s/it]

서울특별시 중랑구 중화동 457
임**


In [40]:
prac_df['소유자'].isna().sum()

24

In [42]:
concat_selected_df = prac_df.copy()

In [43]:
driver.close()

In [44]:
concat_selected_df.reset_index(drop=True, inplace=True)

In [45]:
concat_selected_df.to_csv(bulktrade_basedir+'빠진애들_채워넣음.csv', index=False)

In [46]:
concat_selected_df['소유자'].isna().sum()

24

In [47]:
concat_selected_df.shape

(2097, 15)

In [48]:
concat_selected_df.head()

,지번주소,도로명,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동
0,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,23.55,17000,5,2016.0,2.0,2018-01-04,721.868365,5.0,오피스텔,주식회사하***,강남구,세곡동
1,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,5,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,주식회사하***,강남구,세곡동
2,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,6,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,주식회사하***,강남구,세곡동
3,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,8,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,주식회사하***,강남구,세곡동
4,서울특별시 강남구 세곡동 582-0,헌릉로571길 7,강남레체,28.64,21000,5,2016.0,2.0,2018-01-04,733.240223,5.0,오피스텔,주식회사하***,강남구,세곡동
